In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os
import shutil
from random import shuffle 
from tqdm import tqdm 
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Define functions for logistic classification

def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w, b

def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

def forward_backward_propagation(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients

def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(number_of_iterarion):
        
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 100 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
    
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

def predict(w,b,x_test):
    
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))

    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):

    dimension =  x_train.shape[0]
    w,b = initialize_weights_and_bias(dimension)

    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)
    y_prediction_train = predict(parameters["weight"],parameters["bias"],x_train)
    
    print("Test Accuracy: {} %".format(round(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100,2)))
    print("Train Accuracy: {} %".format(round(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,2)))
    return parameters

In [3]:
predict_images = "prediction_images_renamed"
image_size = 150

def predict_data():
    test_data_bat = [] 
    test_data_no_bat=[]
    for image1 in tqdm(os.listdir(predict_images)): 
        path = os.path.join(predict_images, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        test_data_bat.append(img1) 
    
    test_data= np.asarray(test_data_bat) 
    return test_data 

train_data = predict_data()
print("Processed prediction images")

100%|█████████████████████████████████████████████████████████████████████████████| 3022/3022 [00:08<00:00, 350.47it/s]


Processed prediction images


In [4]:
x_data = train_data
x_data = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [5]:
x_train = x_data
number_of_train = x_train.shape[0]
x_train_flatten = x_train.reshape(number_of_train,x_train.shape[1]*x_train.shape[2])
x_train = x_train_flatten.T

In [18]:
weights = np.loadtxt("weights.csv", delimiter=",")
bias = np.loadtxt("bias.csv", delimiter=",")[0]

In [41]:
w = np.full((x_train.shape[0],1),0.01)
for i in range(0, x_train.shape[0]):
    w[i][0] = weights[i]

[[0.01]
 [0.01]
 [0.01]
 ...
 [0.01]
 [0.01]
 [0.01]]


In [42]:
predictions = predict(w, bias, x_train)

In [43]:
print(predictions)

[[0. 0. 0. ... 0. 0. 0.]]


In [44]:
one = []
i = 1
for num in predictions[0]:
    if num > .5:
        one.append(i)
    i = i + 1

In [ ]:
for name in one:
    src = predict_images + "\\" + str(name) + ".jpg"
    dst = "bat_select\\" + str(name) + ".jpg"
    os.rename(src, dst)